In [1]:
import json
import csv
import requests
import pandas as pd
from config import api_key
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
results_object = []
for x in range(359):
    url ="https://api.data.gov/ed/collegescorecard/v1/schools.json?&_fields=id,school.name,school.accreditor,school.ownership&api_key=" + api_key + "&_page=" + str(x)
    #print(requests.get(url))
    #print(requests.get(url).json())
    response = requests.get(url).json()
    #print(json.dumps(response, indent=4, sort_keys=True))
    results_object = results_object + response["results"]
#table3 = pd.DataFrame(response['results'])
#table3
print(results_object)

[{'school.name': 'New York Theological Seminary', 'school.accreditor': 'Commission on Accrediting of the Association of Theological Schools', 'id': 193894, 'school.ownership': 2}, {'school.name': 'Carver Bible College', 'id': 139287, 'school.ownership': 2, 'school.accreditor': None}, {'school.name': 'Strayer University-Florida', 'school.accreditor': 'Middle States Commission on Higher Education', 'id': 449038, 'school.ownership': 3}, {'school.name': 'Strayer University-Delaware', 'school.accreditor': 'Middle States Commission on Higher Education', 'id': 450298, 'school.ownership': 3}, {'school.name': 'European Academy of Cosmetology and Hairdressing', 'school.accreditor': 'National Accrediting Commission of Career Arts and Sciences', 'id': 457332, 'school.ownership': 3}, {'school.name': 'Tenaj Salon Institute', 'school.accreditor': 'National Accrediting Commission of Career Arts and Sciences', 'id': 457448, 'school.ownership': 3}, {'school.name': 'Advanced Beauty College', 'school.accr

In [3]:
college_table = pd.DataFrame(results_object)
college_table.head()

,id,school.accreditor,school.name,school.ownership
0,193894,Commission on Accrediting of the Association o...,New York Theological Seminary,2
1,139287,None,Carver Bible College,2
2,449038,Middle States Commission on Higher Education,Strayer University-Florida,3
3,450298,Middle States Commission on Higher Education,Strayer University-Delaware,3
4,457332,National Accrediting Commission of Career Arts...,European Academy of Cosmetology and Hairdressing,3


In [4]:
college_table.to_csv("CollegeData.csv")

In [5]:
renamed_df = college_table.rename(columns={"id":"school_id", "school.accreditor":"school_accreditor", "school.name":"school_name", "school.ownership":"private_public"})
renamed_df.head()

,school_id,school_accreditor,school_name,private_public
0,193894,Commission on Accrediting of the Association o...,New York Theological Seminary,2
1,139287,None,Carver Bible College,2
2,449038,Middle States Commission on Higher Education,Strayer University-Florida,3
3,450298,Middle States Commission on Higher Education,Strayer University-Delaware,3
4,457332,National Accrediting Commission of Career Arts...,European Academy of Cosmetology and Hairdressing,3


In [6]:
renamed_df[renamed_df.duplicated(keep=False)]

,school_id,school_accreditor,school_name,private_public


In [7]:
renamed_df.to_csv("CollegeNewData.csv")

In [11]:
rds_connection_string = "root:jaymatadi7@127.0.0.1/school_uni_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [12]:
renamed_df.to_sql("school_name", con = engine, schema=None, if_exists='replace', index=False)